In [148]:
import pandas as pd
import MeCab
import json
from os import listdir
from os.path import isfile, join
import re

In [149]:
japaneseMasculineWordsList = open('./data/gendered_words_lists/ja/masculine.json', 'r')
japaneseMasculineWords = json.load(japaneseMasculineWordsList)
japaneseFeminineWordsList = open('./data/gendered_words_lists/ja/feminine.json', 'r')
japaneseFeminineWords = json.load(japaneseFeminineWordsList)

In [155]:
# Count the number of gendered words in each job advertisement
dataDirectory = './data/job_advertisements/ja/cleaned/'
# get the list of file names
files = [f for f in listdir(dataDirectory) if isfile(join(dataDirectory, f)) and ('failed' not in f)]

print(files[0])
# open job advertisements file
jsonOpen = open(dataDirectory + files[0], 'r')
jobAdvertisements = json.load(jsonOpen)
for jobAdvertisement in jobAdvertisements:
    print(jobAdvertisement['all_sentences'])
    break


pharmacist.json
命を救う「日本赤十字社」で【薬剤師】★九州エリア大募集★ 輸血用血液製剤の安定供給のための増員募集です。「特定生物由来製品」である血液製剤。その製造から販売にいたる各業務は多くの法律や規則に従って行われ専門知識を必要とします。特に「薬剤師」は当血液センターにとって不可欠な存在。医療機関や医師に対する血液製剤の説明や調整業務など薬剤師としてその活躍の場は多岐にわたります。 【日本赤十字社の薬剤師】輸血用血液製剤の製造・検査・品質管理・医薬情報・臨床開発などの業務【具体的には】日本赤十字社が実施する血液事業における「血液製剤の調製」「検査」「品質管理」「医薬情報」「臨床開発」などを薬剤師として幅広く担当。《血液製剤の製造》献血ルームや献血バスで献血された血液をもとに、「輸血用血液製剤」をつくります。《検査》献血された血液の血液型検査、血球計数検査、感染症関連検査など《品質管理》血液製剤や試薬の品質管理、受け入れ試験、関連部署の管理など《医薬情報(学術情報)》血液製剤に関する学術情報の収集、提供、安全管理など 大卒以上■要薬剤師資格■パソコンの基本操作ができる方【具体的には】45歳未満(キャリア形成のため/例外事由3号のイ)業界・職種未経験の方歓迎輸血用血液製剤の原料確保から製造・供給と、血液事業全般を日本赤十字社が担っています。薬剤師資格があっても、全員が未経験スタートのようなものです。まずは研修を通して、業務の進め方を一つひとつ覚えていきましょう!


In [156]:
# Count the number of gendered words in each job advertisement
dataDirectory = './data/job_advertisements/ja/cleaned/'
# get the list of file names
files = [f for f in listdir(dataDirectory) if isfile(join(dataDirectory, f)) and ('failed' not in f)]

# declare the dataframe object to save the results
df = pd.DataFrame(index=['index'], columns=['dataIndex', 'occupation', 'masclineWordCount', 'feminineWordCount', 'wordCount'])
dfByOccupation = pd.DataFrame(index=['index'], columns=['occupation', 'masclineWordCount', 'feminineWordCount', 'wordCount'])
genderedWordDf = pd.DataFrame(index=['index'], columns=['word', 'occupation'])

# initiate the tagger to parse and clean Japanese sentences
tagger = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
tagger.parse('')
re_hiragana = re.compile(r'^[あ-ん]+$')

dataIndex = 1    
for file in files:
    # open job advertisements file
    jsonOpen = open(dataDirectory + file, 'r')
    jobAdvertisements = json.load(jsonOpen)
    occupation = file.replace('.json', '')
    
    masculineWordCounterPerOccupation = 0
    feminineWordCounterPerOccupation = 0
    wordCounterPerOccupation = 0

    dfIndex = 1
    for jobAdvertisement in jobAdvertisements:
        if jobAdvertisement['all_sentences']:
            all_sentences =  jobAdvertisement['all_sentences']
                        
            masculineWordCounterPerAdvertisement = 0
            feminineWordCounterPerAdvertisement = 0
            counter = 0
            
            node = tagger.parseToNode(all_sentences)
            while node:
                counter += 1
                #select specific word types
                if node.feature.split(",")[0] in  ['名詞', '形容詞', '動詞']:
                    if node.feature.split(",")[0] == '': 
                        None
                    elif len(node.feature.split(",")[6]) == 1: 
                        None
                    elif re_hiragana.fullmatch(node.feature.split(",")[6]):
                        None
                    else: 
                        for japaneseMasculineWord in japaneseMasculineWords:
                            if japaneseMasculineWord == node.feature.split(",")[6]:
                                dataIndex += 1
                                masculineWordCounterPerAdvertisement += 1
                                wordData = pd.DataFrame({
                                    'word': japaneseMasculineWord,
                                    'occupation': occupation,
                                }, index=[str(counter)])
                                genderedWordDf = pd.concat([genderedWordDf, wordData])
                                break
                        for japaneseFeminineWord in japaneseFeminineWords:
                            if japaneseFeminineWord == node.feature.split(",")[6]:
                                dataIndex += 1
                                feminineWordCounterPerAdvertisement += 1
                                wordData = pd.DataFrame({
                                    'word': japaneseFeminineWord,
                                    'occupation': occupation,
                                }, index=[str(counter)])
                                genderedWordDf = pd.concat([genderedWordDf, wordData])
                                break
                else:
                    pass
                node = node.next
            # if masculineNumber == 0 and feminineNumber == 0:
            #     x = 0
            # else:
            #     x = (1/(masculineNumber + feminineNumber)) * (masculineNumber - feminineNumber)
            # genderBiasScore = 1 / (1 + math.exp(-x))

            record =  pd.DataFrame({
                'dataIndex': dataIndex,
                'occupation': occupation,
                'masclineWordCount': masculineWordCounterPerAdvertisement,
                'feminineWordCount': feminineWordCounterPerAdvertisement,
                'wordCount': counter
            }, index=[str(dfIndex)])
            df = pd.concat([df, record])
            masculineWordCounterPerOccupation += masculineWordCounterPerAdvertisement
            feminineWordCounterPerOccupation += feminineWordCounterPerAdvertisement
            wordCounterPerOccupation += counter
        dfIndex += 1

    record =  pd.DataFrame({
        'occupation': occupation,
        'masclineWordCount' : masculineWordCounterPerOccupation,
        'feminineWordCount' : feminineWordCounterPerOccupation, 
        'wordCount' : wordCounterPerOccupation
    }, index=[str(dataIndex)])
    dfByOccupation = pd.concat([dfByOccupation, record])

df.to_csv('./result/japanese_result.csv')
dfByOccupation.to_csv('./result/japanese_result_per_occupation.csv')
genderedWordDf.to_csv('./result/japanese_gendered_words_in_advertisements.csv')
